In [ ]:
%matplotlib inline

In [ ]:
import os
import glob
import json
import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def objFromFile(filename):
    with open(filename, 'r') as fobj:
        json_str = fobj.readline()
        fobj.close()
    return json.loads(json_str)

In [ ]:
data_home = os.path.expanduser('~/Documents/Tesla')
folders = glob.glob('{}/2*'.format(data_home))
folders.sort()

t = []
b = []
for folder in folders:
    files = glob.glob('{}/*.json'.format(folder))
    files.sort()
    file = files[-1]
    # print('{}'.format(file))
    data = objFromFile(file)
    o = data['charge_state']
    t.append(datetime.datetime.fromtimestamp(o['timestamp'] / 1000))
    b.append(o['battery_level'])
    print('{} -> {}% {} {}'.format(os.path.basename(file), o['battery_level'], o['charging_state'], t[-1].weekday()))

In [ ]:
# Find the previous Sunday
d = t[0]
oneday = datetime.timedelta(days=1)
while d.weekday() != 6:
    d -= oneday
# display('{}'.format(d.strftime('%Y/%m/%d %H:%M %A')))

# Start time
s = datetime.datetime.strptime(d.strftime('%Y-%m-%d'), '%Y-%m-%d')
# display('{}'.format(s.strftime('%Y/%m/%d %H:%M %A')))

# Group the weeks
tt = []
bb = []
k = 0
while k < len(t):
    tw = []
    bw = []
    for i in range(7):
        tw.append(s)
        if k < len(t) and s <= t[k] and t[k] < (s + oneday):
            bw.append(b[k])
            k += 1
        else:
            bw.append(np.nan)
        s += oneday
    tt.append(tw)
    bb.append(bw)


In [ ]:
import font

# bgColor = (1.0, 1.0, 1.0)
bgColor = (0.77, 0.77, 0.8)

class WeeklyChart:
    def __init__(self, dpi=72, figsize=(1280, 720), t=1.0):
        self._dpi = dpi
        self._figsize = figsize
        self.featureScale = t
        self.fontproperties = font.Properties(scale=t)

        # Use a separate set of context properties
        context_properties = {
            'font.family': 'serif',
            'font.serif': ['Arial', 'DejaVu Serif'],
            'axes.edgecolor': 'white',
            'axes.facecolor': 'black',
            'axes.labelcolor': 'white',
            'axes.linewidth': 1.0 * self.featureScale,
            'axes.titlepad': 8.0 * self.featureScale,
            'grid.color': 'white',
            'hatch.color': 'white',
            'text.color': 'white',
            'xtick.color': 'white',
            'xtick.direction': 'in',
            'xtick.major.pad': 9.0 * self.featureScale,
            'xtick.major.size': 4.0 * self.featureScale,
            'xtick.major.width': 1.0 * self.featureScale,
            'ytick.color': 'white',
            'ytick.direction': 'in',
            'ytick.major.pad': 7.5 * self.featureScale,
        }
        with matplotlib.pyplot.rc_context(context_properties):
            # Create a new figure
            w, h = self._figsize[0], self._figsize[1]    # Axis width & height in points
            figsize = np.array([w, h]) / self._dpi       # Figure size in inches
            self.fig_dat = matplotlib.pyplot.figure(figsize=figsize, dpi=self._dpi, frameon=False)
            self.dat = matplotlib.pyplot.axes([0.0, 0.0, 1.0, 1.0], facecolor=bgColor)
            self.dat.set_xlim((0, 7))
            self.dat.set_ylim((0, 4))
            self.dat.xaxis.set_visible(False)
            self.dat.yaxis.set_visible(False)
            self.dat.spines['top'].set_visible(False)
            self.dat.spines['right'].set_visible(False)
            self.dat.spines['bottom'].set_visible(False)
            self.dat.spines['left'].set_visible(False)
            self.dat.patch.set_facecolor(bgColor)
            self.fig_dat.canvas.draw()

In [ ]:
view = WeeklyChart()

In [ ]:
fig = plt.figure(figsize=(12, 3), dpi=96)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=35)
ax = plt.gca()
xfmt = matplotlib.dates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(xfmt)
# plt.plot(t, b, '.-')
plt.stem(t, b)
plt.grid()

In [ ]:
t = []
b = []
for file in files:
    data = objFromFile(file)
    o = data['drive_state']
    
    o = data['charge_state']
    t.append(datetime.datetime.fromtimestamp(o['timestamp'] / 1000))
    b.append(o['battery_level'])
    print('{} -> {}% {}'.format(os.path.basename(file), o['battery_level'], o['charging_state']))

In [ ]:
fig = plt.figure(figsize=(12, 3), dpi=96)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=25)
ax = plt.gca()
xfmt = matplotlib.dates.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)
# plt.plot(t, b, '.-')
plt.stem(t, b)

In [ ]:
for file in files[-5:]:
    data = objFromFile(file)
    display(file)
    display(data['drive_state'])